In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import glob
import datetime
import numba

from scipy import odr

import bokeh
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import gridplot, row, column
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()
import holoviews as hv
import bebi103
hv.extension('bokeh')



Using same techniques as NOAA_flask.ipynb used in propane bayes analysis, except need to include another var from the raw file 

Also this will analyze the new noaa data beyond 2018 (when i get it) 

Here is where i accessed data beyond 2018 for c3, c2, https://gml.noaa.gov/aftp/data/trace_gases/ 
This is where I got ch4, co2, and co: https://gml.noaa.gov/aftp/data/trace_gases/ch4/pfp/ 
this is wrong directory: https://gml.noaa.gov/aftp/data/trace_gases/voc/c3h8/flask/surface/ 

In [ ]:
def open_noaa_txt_num2(filename_as_string):
    with open(filename_as_string, 'r') as f:       
        f_list = f.read()
        f_list = f_list.split('\n')
        number_of_header_lines = int(f_list[0].split(' ')[-1])
        
        header_names = f_list[number_of_header_lines-1].split(' ')
#         print(header_names)
        data_body = f_list[number_of_header_lines:-1]
        
        data = []
        for i in range(len(data_body)):
            data.append(" ".join(data_body[i].split()).split(' '))
        
        return header_names, data

so c2, c3, and co have the same headers. ch4 and co2 have value_uncertainty which is an additional var compared to the others

In [ ]:
file_paths = '/Users/arianatribby/git/oklahoma_propane/data/NOAA_sgp/*.txt'
data_body = []
file_names = sorted(glob.glob(file_paths))

for i in range(len(file_names)):
    site_header,site_data = open_noaa_txt_num2(file_names[i])
    var_name = file_names[i].split('/')[-1].split('_')[0]
    [x.append(var_name) for x in site_data]
    data_body.append(site_data)

    
flat_data_body = [item for sublist in data_body for item in sublist]
site_header.append('parameter_formula')


In [ ]:
df_noaap1 = pd.DataFrame(flat_data_body, columns=site_header)
df_noaap1.tail()

first, add an uncertainty column as na 

In [ ]:
df_noaap1['value_unc'] = np.empty(len(df_noaap1.day.values)) * np.nan

Now read in the other files

In [ ]:
file_paths = '/Users/arianatribby/git/oklahoma_propane/data/NOAA_sgp/headertype2/*.txt'
data_body = []
file_names = sorted(glob.glob(file_paths))

for i in range(len(file_names)):
    site_header,site_data = open_noaa_txt_num2(file_names[i])
    var_name = file_names[i].split('/')[-1].split('_')[0]
    [x.append(var_name) for x in site_data]
    data_body.append(site_data)
    
flat_data_body = [item for sublist in data_body for item in sublist]
site_header.append('parameter_formula')


In [ ]:
df_noaap2 = pd.DataFrame(flat_data_body, columns=site_header)
df_noaap2.tail()

Combine the dfs with the same columns 

In [ ]:
NOAA_df = pd.concat([df_noaap1,df_noaap2])

In [ ]:
NOAA_df.loc[NOAA_df['parameter_formula'] == 'C2H6'].tail()

## Quality Control 

In the folders where you read in the files, are readme files. There is a section on quality control.

NOAA ESRL uses a 3-column quality control flag where each column
is defined as follows:
```
column 1    REJECTION flag.  An alphanumeric other
            than a period (.) in the FIRST column indicates
            a sample with obvious problems during collection
            or analysis.  This measurement should not be interpreted.

column 2    SELECTION flag.  An alphanumeric other than a
            period (.) in the SECOND column indicates a sample
            that is likely valid but does not meet selection
            criteria determined by the goals of a particular
            investigation.

column 3    INFORMATION flag.  An alphanumeric other than a period (.) 
            in the THIRD column provides additional information 
            about the collection or analysis of the sample.

            WARNING: A "P" in the 3rd column of the QC flag indicates
            the measurement result is preliminary and has not yet been 
            carefully examined by the PI.  The "P" flag is removed once 
            the quality of the measurement has been assessed.

            Flag         Description

RETAINED    ...         (3 periods) Good sampling and analysis.


            ..0         (3rd column zero) Preliminary data awaiting QC
                        

SELECTED    .A.         Suspected analysis issue that may affect data quality,
			although the affect may not be readily evident. Use with caution.

            .N.         suspected sampling issue that may affect data quality,
            	        although the affect may not be apparent. Use with caution.

REJECTED    A..         Known analysis issue that has compromised data quality.
			Do not use these data!

            N..         Known sampling issue that has compromised data quality.
			Do not use these data
```

For ccgg flask: 

```
# qcflag:provider_comment : This is the NOAA 3-character quality control flag.  Column 1 is the REJECTION flag. An alphanumeric other than a period (.) in the FIRST column indicates a sample with obvious problems during collection or analysis. This measurement should not be interpreted. Column 2 is the SELECTION flag. An alphanumeric other than a period (.) in the SECOND column indicates a sample that is likely valid but does not meet selection criteria determined by the goals of a particular investigation. For example, it might not have been obtained during 'background' conditions.  Column 3 is the INFORMATION flag. An alphanumeric other than a period (.) in the THIRD column provides additional information about the collection or analysis of the sample. A P in the 3rd column of the QC flag indicates the measurement result is preliminary and has not yet been carefully examined by the PI.  The P flag is removed once the quality of the measurement has been determined.

```

I will keep '...', '..P'. 

In [ ]:
NOAA_df = NOAA_df.loc[NOAA_df['qcflag'].isin(['...','..P'])].reset_index()

In [ ]:
NOAA_df.columns

### Create UTC time

In [ ]:
newtime = []
newdatetime = []
for i in range(len(NOAA_df)):
    newtime.append(datetime.datetime(int(NOAA_df['year'].values[i]),int(NOAA_df['month'].values[i]),
                      int(NOAA_df['day'].values[i]),int(NOAA_df['hour'].values[i]),
                      int(NOAA_df['minute'].values[i]),int(NOAA_df['second'].values[i])).timestamp())
    newdatetime.append(datetime.datetime(int(NOAA_df['year'].values[i]),int(NOAA_df['month'].values[i]),
                      int(NOAA_df['day'].values[i]),int(NOAA_df['hour'].values[i]),
                      int(NOAA_df['minute'].values[i]),int(NOAA_df['second'].values[i])))

NOAA_df['dt_time'] = newdatetime
NOAA_df['completetime'] = newtime

Convert measurements from string to numbers

In [ ]:
NOAA_df['value'] = [float(x) for x in NOAA_df.copy().value.values]

In [ ]:
NOAA_df['completetime'][0]

### Find common timepoint measurements 

Unlike previous noaa datasets, you only have one site, sgp. So the c3 and c2 have exactly the same time points. However, ch4, co and co2 do not have the same time points. Need to know where those overlap with c3 and c2. 

In [ ]:
NOAA_df.parameter_formula.unique()

In [ ]:
test1 = NOAA_df.loc[(NOAA_df['parameter_formula'] == 'C2H6')][['value','completetime','year','day']]
test2 = NOAA_df.loc[(NOAA_df['parameter_formula'] == 'C3H8')][['value','completetime','year','day']]
test3 = NOAA_df.loc[(NOAA_df['parameter_formula'] == 'ch4')][['value','completetime','year','day']]
test4 = NOAA_df.loc[(NOAA_df['parameter_formula'] == 'co')][['value','completetime','year','day']]
test5 = NOAA_df.loc[(NOAA_df['parameter_formula'] == 'co2')][['value','completetime','year','day']]

In [ ]:
print(len(test1))
print(len(test2))
print(len(test3))
print(len(test4))
print(len(test5))

But even thought C2 and C3 are the same length, they don't always have exactly the same number of measurements for certain time points. For example, see the printed example below. Also, there are duplicate measurements at the same time point (including the seconds) for all the species (same example). 

Going to group by the time for each species individually, take the median, then paste everything back into one dataframe. Then, will get intersection of the time for all species at once using groupby again. 

In [ ]:
max(NOAA_df.loc[NOAA_df['parameter_formula'] == 'co'].groupby(['dt_time']).count().year)

It looks like time isn't repeated within a single species

Before you can use groupby, though, you need to convert all columns that you plan on keeping into floats or ints, otherwise, those columns are ignored when you use aggregate functions on them. Parameter_formula and completetime are already floats 

In [ ]:
NOAA_df.columns

In [ ]:
NOAA_df['year'] = [int(x) for x in NOAA_df['year'].values]
NOAA_df['month'] = [int(x) for x in NOAA_df['month'].values]
NOAA_df['day'] = [int(x) for x in NOAA_df['day'].values]
NOAA_df['hour'] = [int(x) for x in NOAA_df['hour'].values]
NOAA_df['minute'] = [int(x) for x in NOAA_df['minute'].values]
NOAA_df['second'] = [int(x) for x in NOAA_df['second'].values]
NOAA_df['value_unc'] = [float(x) for x in NOAA_df['value_unc'].values]
NOAA_df['latitude'] = [float(x) for x in NOAA_df['latitude'].values]
NOAA_df['longitude'] = [float(x) for x in NOAA_df['longitude'].values]
NOAA_df['altitude'] = [float(x) for x in NOAA_df['altitude'].values]
NOAA_df['elevation'] = [float(x) for x in NOAA_df['elevation'].values]
NOAA_df['intake_height'] = [float(x) for x in NOAA_df['intake_height'].values]

In [ ]:
NOAA_df.year

The following is done if there are repeat measurements within a single measurement. This is the case for co, co2, and ch4. I'll just keep the code for all variables for ease since some of the other variables for those 3 parameters will be lost (can't get the median of non float or int things)

Extract variables you intend to keep for groupby

In [ ]:
c3df = NOAA_df.loc[NOAA_df['parameter_formula'] == 'C3H8'][["year","month","day","hour","minute","second",
                                                           "value","value_unc","latitude","longitude",
                                                            "altitude","elevation","intake_height","dt_time","completetime"]]
c2df = NOAA_df.loc[NOAA_df['parameter_formula'] == 'C2H6'][["year","month","day","hour","minute","second",
                                                           "value","value_unc","latitude","longitude",
                                                            "altitude","elevation","intake_height","dt_time","completetime"]]
codf = NOAA_df.loc[NOAA_df['parameter_formula'] == 'co'][["year","month","day","hour","minute","second",
                                                           "value","value_unc","latitude","longitude",
                                                            "altitude","elevation","intake_height","dt_time","completetime"]]
co2df = NOAA_df.loc[NOAA_df['parameter_formula'] == 'co2'][["year","month","day","hour","minute","second",
                                                           "value","value_unc","latitude","longitude",
                                                            "altitude","elevation","intake_height","dt_time","completetime"]]
ch4df = NOAA_df.loc[NOAA_df['parameter_formula'] == 'ch4'][["year","month","day","hour","minute","second",
                                                           "value","value_unc","latitude","longitude",
                                                            "altitude","elevation","intake_height","dt_time","completetime"]]

In [ ]:
c3_time_red = c3df.groupby(['dt_time']).median().reset_index()
c2_time_red = c2df.groupby(['dt_time']).median().reset_index()
ch4_time_red = ch4df.groupby(['dt_time']).median().reset_index()
co_time_red = codf.groupby(['dt_time']).median().reset_index()
co2_time_red = co2df.groupby(['dt_time']).median().reset_index()

Now put all the variables together again into an untidy dataset

In [ ]:
c3_time_red['parameter'] = ['c3h8' for x in range(len(c3_time_red))]
c2_time_red['parameter'] = ['c2h6' for x in range(len(c2_time_red))]
ch4_time_red['parameter'] = ['ch4' for x in range(len(ch4_time_red))]
co_time_red['parameter'] = ['co' for x in range(len(co_time_red))]
co2_time_red['parameter'] = ['co2' for x in range(len(co2_time_red))]
noaa_time_red = pd.concat([c3_time_red,c2_time_red,ch4_time_red,co_time_red,co2_time_red], ignore_index=True)

In [ ]:
noaa_time_red.loc[(noaa_time_red['parameter'] == 'c3h8') & (noaa_time_red['year'] == 2017)].head()

Now do a new groupby where you find cases where time is repeated 5 times (for the 5 variables)

In [ ]:
max(noaa_time_red.groupby(['dt_time']).count().year)

The result is each variable contains the count number. So count function was applied to each variable. But what you really need is the index. The issue is that the count function is applied even to the index variable. So can you then figure out which dt_times correspond to counts greater than 5.

In [ ]:
time_counts = noaa_time_red.groupby(['dt_time'])['year'].count().reset_index()

In [ ]:
len(time_counts.loc[time_counts['year'] == 5])

In [ ]:
intersecting_times = time_counts.loc[time_counts['year'] == 5]['dt_time']

In [ ]:
intersecting_times

In [ ]:
len(noaa_time_red.loc[noaa_time_red['dt_time'].isin(intersecting_times)])

Remember that the df is not tidy, so it would be 732 multiplied by 5

In [ ]:
732*5

Great, save out that dataset! 

In [ ]:
noaa_time_intersected = noaa_time_red.loc[noaa_time_red['dt_time'].isin(intersecting_times)].reset_index()

In [ ]:
noaa_time_intersected.head()

In [ ]:
len(noaa_time_red.loc[noaa_time_red['dt_time'].isin(intersecting_times)])

In [ ]:
noaa_time_red.parameter.unique()

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')
dc3 = noaa_time_red.loc[noaa_time_red['parameter'] == 'c3h8']
dc2 = noaa_time_red.loc[noaa_time_red['parameter'] == 'c2h6']


p.circle(dc3.dt_time.values, dc3.value.values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='c3h8')

p.circle(dc2.dt_time.values, dc2.value.values, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='c2h6')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "CxCx ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)


Note that it does show 2017 values, it is just that you don't have any for early 2017. 

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')
dc3 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'c3h8']
dc2 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'c2h6']


p.circle(dc3.dt_time.values, dc3.value.values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='c3h8')

p.circle(dc2.dt_time.values, dc2.value.values, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='c2h6')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "CxCx ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

In [ ]:
len(noaa_time_intersected.loc[(noaa_time_intersected['year'] == 2017) * (noaa_time_intersected['parameter'] == 'c2h6')])

fresh emission and fire filter, see below

Fire, september 13, 14, 2017 not sure about end date 
https://wildfiretoday.com/2017/09/14/rankin-fire-expands-to-over-1000-acres-while-the-beaver-fire-grows-nearby/ 



### Now filter for fresh emissions based on analysis from NOAA_flask.ipynb 

First make everything into a tidy dataset 

In [ ]:
df_loc_c2h6 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'c2h6']
df_loc_c3h8 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'c3h8']
df_loc_ch4 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'ch4']
df_loc_co = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'co']
df_loc_co2 = noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'co2']


df_loc_c2h6 = df_loc_c2h6.drop(columns=['parameter'])
df_loc_c3h8 = df_loc_c3h8.drop(columns=['parameter'])
df_loc_ch4 = df_loc_ch4.drop(columns=['parameter'])
df_loc_co = df_loc_co.drop(columns=['parameter'])
df_loc_co2 = df_loc_co2.drop(columns=['parameter'])


In [ ]:
NOAA_paramaters = pd.DataFrame()
NOAA_paramaters['dt_time'] = df_loc_c2h6.copy().dt_time.values
NOAA_paramaters['year'] = df_loc_c2h6.copy().year.values
NOAA_paramaters['month'] = df_loc_c2h6.copy().month.values
NOAA_paramaters['day'] = df_loc_c2h6.copy().day.values
NOAA_paramaters['hour'] = df_loc_c2h6.copy().hour.values
NOAA_paramaters['minute'] = df_loc_c2h6.copy().minute.values
NOAA_paramaters['second'] = df_loc_c2h6.copy().second.values
NOAA_paramaters['completetime'] = df_loc_c2h6.copy().completetime.values


NOAA_paramaters['c2h6'] = df_loc_c2h6.copy().value.values
NOAA_paramaters['c3h8'] = df_loc_c3h8.copy().value.values
NOAA_paramaters['ch4'] = df_loc_ch4.copy().value.values
NOAA_paramaters['co'] = df_loc_co.copy().value.values
NOAA_paramaters['co2'] = df_loc_co2.copy().value.values
NOAA_paramaters['c2h6_unc'] = df_loc_c2h6.copy().value_unc.values
NOAA_paramaters['c3h8_unc'] = df_loc_c3h8.copy().value_unc.values
NOAA_paramaters['ch4_unc'] = df_loc_ch4.copy().value_unc.values
NOAA_paramaters['co_unc'] = df_loc_co.copy().value_unc.values
NOAA_paramaters['co2_unc'] = df_loc_co2.copy().value_unc.values

# I double checked lat/lon/alt are the same between the different variables 
NOAA_paramaters['lat'] = df_loc_co2.copy().latitude.values
NOAA_paramaters['lon'] = df_loc_co2.copy().longitude.values
NOAA_paramaters['altitude'] = df_loc_co2.copy().altitude.values
NOAA_paramaters['elevation'] = df_loc_co2.copy().elevation.values
NOAA_paramaters['intake_height'] = df_loc_co2.copy().intake_height.values

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='', x_axis_type="log", y_axis_type="log")
colors = bokeh.palettes.d3['Category20'][20]


p.circle(NOAA_paramaters.c2h6*1e-3, 
         NOAA_paramaters.c3h8*1e-3, 
         color=colors[0], size=5, fill_alpha=.5, legend_label = 'SGP')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'

p.legend.location = "bottom_right"
bokeh.io.show(p)

Data below 1 ppb c3h8 is aged air. Just going to go off of your paper. 

In [ ]:
def pertcile_score(array,percentile):
    """ Array is a list of gas measurements. 
    Percentile is the value you want to calculate
    the score for. 
    """
    return st.stats.scoreatpercentile(array, percentile)


In [ ]:
cutoff_conc = pertcile_score(NOAA_paramaters.c3h8.values*1e-3, 50)

In [ ]:
cutoff_conc

I feel that is too low. Just going to use 1 ppb as the cutoff.

In [ ]:
cutoff_conc = 1.

In [ ]:
NOAA_fresh = NOAA_paramaters.loc[NOAA_paramaters['c3h8'] >= 1000.]

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.c3h8.values*1e-3, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='c3h8')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.c2h6.values*1e-3, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='c2h6')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "CxCx ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

What about fires: check co and co2

In [ ]:
NOAA_fresh.co

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(NOAA_fresh.co.values, NOAA_fresh.co2.values, size=5)


p.xaxis.axis_label = "co"
p.yaxis.axis_label = "co2 "
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.co.values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='co')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.co2.values, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='co2')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "C "
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
p.legend.location = "center"
bokeh.io.show(p)

These don't really look impacted in terms of fire emiss? You can remove values greater than 200 CO. 

What are the average number of measurements each month for the relevant summer months

In [ ]:
NOAA_fresh_summer = NOAA_fresh.loc[(NOAA_fresh['month'] == 6) | (NOAA_fresh['month'] == 7) | 
               (NOAA_fresh['month'] == 8) | (NOAA_fresh['month'] == 9)]

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

# the x axis is the first instance of time for the groupby (grouping per month)
p.circle(NOAA_fresh_summer.groupby(['year','month'])['dt_time'].describe()['first'].values, 
         NOAA_fresh_summer.groupby(['year','month'])['dt_time'].count().values, 
         size=5, fill_alpha=.5, color=colors[0])

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "number of data (counted each month)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

In [ ]:
NOAA_fresh_summer.groupby(['year','month'])['dt_time'].describe()['first'].values

Compare this new data to the old data for 2017 (the data that you used for the propane bayes paper) 

In [ ]:
old_noaa_data = pd.read_csv('../../../propane_bayes_local/data/processing/NOAAfreshemissions.csv')

In [ ]:
old_noaa_data2017 = old_noaa_data.iloc[1354:1422].reset_index()

In [ ]:
old_noaa_data2017_summer = old_noaa_data2017.iloc[18:49]

In [ ]:
print(len(old_noaa_data2017_summer))
print(len(NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017]))


In [ ]:


fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(old_noaa_data2017_summer.c2h6.values, 
         old_noaa_data2017_summer.c3h8.values, 
         size=5, color=colors[1],legend_label='old data')

p.circle(NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017].c2h6.values, 
         NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017].c3h8.values, 
         size=5, color=colors[0],legend_label='new data')


p.xaxis.axis_label = "c2h6"
p.yaxis.axis_label = "c3h8"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

## Would there be more data if you didn't try to include CO and CO2? 

In [ ]:
noaa_time_red2 = pd.concat([c3_time_red,c2_time_red,ch4_time_red], ignore_index=True)

In [ ]:
time_counts = noaa_time_red2.groupby(['dt_time'])['year'].count().reset_index()

In [ ]:
len(time_counts.loc[time_counts['year'] == 3])

In [ ]:
intersecting_times = time_counts.loc[time_counts['year'] == 3]['dt_time']

In [ ]:
intersecting_times

In [ ]:
len(noaa_time_red2.loc[noaa_time_red2['dt_time'].isin(intersecting_times)])

Remember that the df is not tidy, so it would be multiplied by 3

In [ ]:
len(intersecting_times)*3

Great, save out that dataset! 

In [ ]:
noaa_time_intersected2 = noaa_time_red2.loc[noaa_time_red2['dt_time'].isin(intersecting_times)].reset_index()

In [ ]:
print(len(noaa_time_intersected2.loc[noaa_time_intersected2['parameter'] == 'c3h8']))
print(len(noaa_time_intersected.loc[noaa_time_intersected['parameter'] == 'c3h8']))

Great!

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')
dc3 = noaa_time_red2.loc[noaa_time_red2['parameter'] == 'c3h8']
dc2 = noaa_time_red2.loc[noaa_time_red2['parameter'] == 'c2h6']


p.circle(dc3.dt_time.values, dc3.value.values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='c3h8')

p.circle(dc2.dt_time.values, dc2.value.values, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='c2h6')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "CxCx ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)



In [ ]:
noaa_time_intersected2 = noaa_time_red2.loc[noaa_time_red2['dt_time'].isin(intersecting_times)].reset_index()

In [ ]:
df_loc_c2h6 = noaa_time_intersected2.loc[noaa_time_intersected2['parameter'] == 'c2h6']
df_loc_c3h8 = noaa_time_intersected2.loc[noaa_time_intersected2['parameter'] == 'c3h8']
df_loc_ch4 = noaa_time_intersected2.loc[noaa_time_intersected2['parameter'] == 'ch4']


df_loc_c2h6 = df_loc_c2h6.drop(columns=['parameter'])
df_loc_c3h8 = df_loc_c3h8.drop(columns=['parameter'])
df_loc_ch4 = df_loc_ch4.drop(columns=['parameter'])


In [ ]:
NOAA_paramaters = pd.DataFrame()
NOAA_paramaters['dt_time'] = df_loc_c2h6.copy().dt_time.values
NOAA_paramaters['year'] = df_loc_c2h6.copy().year.values
NOAA_paramaters['month'] = df_loc_c2h6.copy().month.values
NOAA_paramaters['day'] = df_loc_c2h6.copy().day.values
NOAA_paramaters['hour'] = df_loc_c2h6.copy().hour.values
NOAA_paramaters['minute'] = df_loc_c2h6.copy().minute.values
NOAA_paramaters['second'] = df_loc_c2h6.copy().second.values
NOAA_paramaters['completetime'] = df_loc_c2h6.copy().completetime.values


NOAA_paramaters['c2h6'] = df_loc_c2h6.copy().value.values
NOAA_paramaters['c3h8'] = df_loc_c3h8.copy().value.values
NOAA_paramaters['ch4'] = df_loc_ch4.copy().value.values
NOAA_paramaters['c2h6_unc'] = df_loc_c2h6.copy().value_unc.values
NOAA_paramaters['c3h8_unc'] = df_loc_c3h8.copy().value_unc.values
NOAA_paramaters['ch4_unc'] = df_loc_ch4.copy().value_unc.values

# I double checked lat/lon/alt are the same between the different variables 
NOAA_paramaters['lat'] = df_loc_c2h6.copy().latitude.values
NOAA_paramaters['lon'] = df_loc_c2h6.copy().longitude.values
NOAA_paramaters['altitude'] = df_loc_c2h6.copy().altitude.values
NOAA_paramaters['elevation'] = df_loc_c2h6.copy().elevation.values
NOAA_paramaters['intake_height'] = df_loc_c2h6.copy().intake_height.values

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='', x_axis_type="log", y_axis_type="log")
colors = bokeh.palettes.d3['Category20'][20]


p.circle(NOAA_paramaters.c2h6*1e-3, 
         NOAA_paramaters.c3h8*1e-3, 
         color=colors[0], size=5, fill_alpha=.5, legend_label = 'SGP')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'

p.legend.location = "bottom_right"
bokeh.io.show(p)

Data below 1 ppb c3h8 is aged air. Just going to go off of your paper. 

In [ ]:
NOAA_fresh = NOAA_paramaters.loc[NOAA_paramaters['c3h8'] >= 1000.]

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.c3h8.values*1e-3, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='c3h8')

p.circle(NOAA_fresh.dt_time.values, NOAA_fresh.c2h6.values*1e-3, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='c2h6')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "CxCx ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

compare to old data again

In [ ]:
NOAA_fresh_summer = NOAA_fresh.loc[(NOAA_fresh['month'] == 6) | (NOAA_fresh['month'] == 7) | 
               (NOAA_fresh['month'] == 8) | (NOAA_fresh['month'] == 9)]

In [ ]:
print(len(old_noaa_data2017_summer))
print(len(NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017]))


In [ ]:


fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(old_noaa_data2017_summer.c2h6.values, 
         old_noaa_data2017_summer.c3h8.values, 
         size=5, color=colors[1],legend_label='old data')

p.circle(NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017].c2h6.values, 
         NOAA_fresh_summer.loc[NOAA_fresh_summer['year'] == 2017].c3h8.values, 
         size=5, color=colors[0],legend_label='new data')


p.xaxis.axis_label = "c2h6"
p.yaxis.axis_label = "c3h8"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

In [ ]:
NOAA_fresh_summer.groupby(['year','month'])['dt_time'].describe()['first'].values

Yes, keeping this dataset. 

do the bootstrap for c3 vs c2 

In [ ]:
NOAA_fresh_summer.to_csv('../../data/processing/newNOAA_freshemiss2016to2020.csv', index=None)

Also save out data for summer only but accounting for the background 

In [ ]:
NOAA_summer = NOAA_paramaters.loc[(NOAA_paramaters['month'] == 6) | (NOAA_paramaters['month'] == 7) | 
               (NOAA_paramaters['month'] == 8) | (NOAA_paramaters['month'] == 9)]

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='', x_axis_type="log", y_axis_type="log")
colors = bokeh.palettes.d3['Category20'][20]


p.circle(NOAA_summer.c2h6*1e-3, 
         NOAA_summer.c3h8*1e-3, 
         color=colors[0], size=5, fill_alpha=.5, legend_label = 'all')
p.circle(NOAA_fresh_summer.c2h6*1e-3, 
         NOAA_fresh_summer.c3h8*1e-3, 
         color=colors[1], size=5, fill_alpha=.5, legend_label = 'fresh only')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'

p.legend.location = "bottom_right"
bokeh.io.show(p)

In [ ]:
NOAA_summer.to_csv('../../data/processing/newNOAA_oldandfreshemiss2016to2020.csv', index=None)